In [ ]:
from collections import OrderedDict
from datetime import datetime
import logging
import os
from pprint import pprint

import geojson
import geopandas as gpd
import fiona
import matplotlib as plt
import pyodbc
from shapely.geometry import (
    LineString, mapping, shape
)
from shapely.ops import (
    split as shapely_split, transform
)

from python_gis.poc.landgrid_processor import LandgridProcessor
from python_gis.poc.io.pgsql import PgWriter
from python_gis.poc.io.mssql import MsWriter
from python_gis.poc.io.shapefile import (
    read_shapefile
)
from python_gis.poc.tools.spatial import (
    remove_holes, fix_anti_meridian
)
import python_gis.poc.util.log_config

logger = logging.getLogger(__name__)

In [ ]:
!ls $DIML_HOME

In [ ]:
# Ms Sql
ms_driver = os.environ["MSSQL_DRIVER"]
ms_host = os.environ["MSSQL_SERVER"]
ms_db = os.environ["MSSQL_DATABASE"]
ms_user = os.environ["MSSQL_UID"]
ms_pwd = os.environ["MSSQL_PWD"]

# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

# WGS84 (epsg:4326)
gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')
ddl_path = os.path.join(os.environ["DIML_HOME"], 'database', 'mssql', 'schema.sql')
out_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out')
idx_path = os.path.join(os.environ["DIML_HOME"], 'database', "indexes.sql")
test_path = os.path.join(os.environ["DATA_DIR"], 'mssql_test')

## Test Harness (PostgreSQL)

In [ ]:
%%time

with PgWriter(host=pg_host, database=pg_db, user=pg_user,
              password=pg_pwd) as db:

        logger.info(f"Creating database schema...")

        db.execute_script(ddl_path, auto_commit=True)

        logger.info(f"Running landgrid ETL...")

        landgrid = LandgridProcessor(gdb_path, out_path, db)
        landgrid.to_shapefiles(redo_grid=True)
        landgrid.run_transforms()
        landgrid.to_datastore()

        logger.info(f"Building spatial indexes...")

        db.execute_script(idx_path, auto_commit=True)
    

## Test Harness (MS SQL)

In [ ]:
%%time

with MsWriter(driver=os.environ["MSSQL_DRIVER"],
              host=os.environ["MSSQL_SERVER"],
              db=os.environ["MSSQL_DATABASE"],
              uid=os.environ["MSSQL_UID"],
              pwd=os.environ["MSSQL_PWD"]) as db:

    logger.info(f"Creating database schema...")

    db.execute_script(ddl_path, auto_commit=True)  # TODO: add if_exists skip.

    logger.info(f"Running landgrid ETL...")

    landgrid = LandgridProcessor(gdb_path, out_path, db)
#     landgrid.to_shapefiles(redo_grid=False)
    landgrid.run_transforms()
    landgrid.to_datastore()

    # logger.info(f"Building spatial indexes...")
    #
    # db.execute_script(idx_path, auto_commit=True)